In [1]:
import pandas as pd
import importlib
from lib import kpi

In [19]:
importlib.reload(kpi)

<module 'lib.kpi' from 'c:\\workspace\\GitHub\\arcsaef\\lib\\kpi.py'>

In [20]:
data           = kpi.load_data()
rpt_config     = kpi.get_rpt_args()
saef_library   = kpi.get_saef_library()
responses_json = kpi.split_response(data[0])
ppl_collection = kpi.person_construct(responses_json, rpt_config[0])
proj_saef      = kpi.project_construct(responses_json, ppl_collection[0])
buckets        = kpi.matched_library(saef_library, ppl_collection[1])
templates      = kpi.load_templates()
meta_bucket    = buckets[0]
bucket         = buckets[1]
ppl_saef       = dict(sorted(ppl_collection[0].items(), key = lambda x: x[1].get('LastName')))
ppl_hash       = ppl_collection[1]
bulk_responses = data[0]
biblio         = data[1]
scopus         = data[2]
rpt_year       = rpt_config[0]
organisations  = rpt_config[1]
org_shortnames = list(organisations.keys())
proj_saef_nohold = proj_saef[proj_saef.Status != 'On hold']

In [21]:
# 1.remove empty rows
bucket = bucket[bucket['id_person'].isna() == False]
# 2. Make parsing more straight forward
for prsn in ppl_saef:
    if ppl_saef[prsn]['Gender'] == 'Non-binary/Gender diverse' or  ppl_saef[prsn]['Gender']  == 'Prefer not to say':
        ppl_saef[prsn]['Gender'] = 'Other'
# 3. Exclude: Nicole Webster, 027E2DEA-DB06-3946-B8C8-E053EF8E09F0
del ppl_saef['027E2DEA-DB06-3946-B8C8-E053EF8E09F0']; del ppl_hash['NicoleWebster']
# 4. Remove leading/trialing whitespace from  blibliographic entry.
biblio['Biblio'] = biblio['Biblio'].str.strip()
# 5. Jump into your pedantry
for prsn in ppl_saef:
    if ppl_saef[prsn]['Position'] != 'PhD Student' and  ppl_saef[prsn]['Position'] != 'Masters Student' and \
       ppl_saef[prsn]['Position'] != 'Honours Student':
        ppl_saef[prsn]['StudentProjectTitle'] = 'Not applicable'

In [22]:
def condition(x):
    if ppl_saef[x]['Position'] == 'Chief Investigator':
        return 'Chief Investigator'
    elif ppl_saef[x]['Position'] == 'Partner Investigator':
        return 'Partner Investigator'
    elif ppl_saef[x]['Position'] == 'Associate Investigator':
        return 'Associate Investigator'
bucket.loc[:, ('position')] =  bucket['id_person'].apply(condition)

C:\Users\okpokam\AppData\Local\Temp\ipykernel_22488\3836866019.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bucket.loc[:, ('position')] =  bucket['id_person'].apply(condition)


In [23]:
prsn_list = []
for prsn in ppl_saef:
    if ppl_saef[prsn]['Position'] in ['Chief Investigator', 'Partner Investigator', 'Associate Investigator']:
        prsn_list.append(prsn)

# journals authored by AI's, CI's or PI's
prsn_jnls_df = bucket.loc[bucket.itemType.isin(['journalArticle']) & (bucket.id_person.isin(prsn_list)), \
                          ['title', 'name', 'id_person', 'position']]
# all person id's of CI'S & PI's without an authored publication
ci_pi_wo_any_jnls_ids = pd.Series(prsn_list)[~pd.Series(prsn_list).isin(prsn_jnls_df.id_person)]

prsn_no_jnls = []
for prsn in ppl_saef:
    if ppl_saef[prsn]['State'] == 'Active':
        if prsn in ci_pi_wo_any_jnls_ids.to_list():
            prsn_no_jnls.append({'title': 'zzDummy', 'name': ppl_saef[prsn]['FirstName']+ppl_saef[prsn]['LastName'],  'id_person': prsn, 'position': ppl_saef[prsn]['Position'] } )
prsn_no_jnls_df = pd.DataFrame(prsn_no_jnls)

In [24]:
# write journal output by Associate Investigator, Chief Investigator, Partner Investigator to spreadsheet
pd.crosstab(prsn_jnls_df.title, [prsn_jnls_df.position, prsn_jnls_df.name]).to_excel("output/2024/org/pub_auth_crosstab.xlsx", sheet_name=f"{'2024'}")

In [28]:
# output all AI's, CI"S & PI's that are not named on at least 1 journal article in 2024
pd.set_option('display.max_columns', 50)
pd.crosstab(prsn_no_jnls_df.title, [prsn_no_jnls_df.position, prsn_no_jnls_df.name])

position     Associate Investigator                                          \
name     AlejandroFontan Villacampa EmmaCooper FlaviaFerrari FrancescoRicci   
title                                                                         
zzDummy                           1          1             1              1   

position                                                               \
name     GavinWinter GrantDuffy JacintaHolloway-Brown JohannaTurnbull   
title                                                                   
zzDummy            1          1                     1               1   

position                                                                  \
name     KateThompson MehrdadAmirghasemi Pok Man (Bob)Leung ThomSaunders   
title                                                                      
zzDummy             1                  1                  1            1   

position              Chief Investigator                           \
name     ThomasRoland       EmmaJohnston GraemeClark JanStrugnell   
title                                                               
zzDummy             1                  1           1            1   

position                                                              \
name     JonathanRhodes KerrieMengersen KerrieWilson MatthewDunbabin   
title                                                                  
zzDummy               1               1            1               1   

position                        Partner Investigator                         \
name     NoelCressie StuartKaye       CeridwenFraser DanCharman DonaldCowan   
title                                                                         
zzDummy            1          1                    1          1           1   

position                                                             \
name     Eun-PaLim GustavoZuniga IlyaMaclean JaumeFlexas JodieSmith   
title                                                                 
zzDummy          1             1           1           1          1   

position                                                         \
name     JonnyStark KarinaMeredith KrystynaSaunders MardiMcNeil   
title                                                             
zzDummy           1              1                1           1   

position                                                                   \
name     MargaretBarbour MatthewWheeler MiguelOlalla-Tárraga NeridaWilson   
title                                                                       
zzDummy                1              1                    1            1   

position                                                            
name     QuanHua Reka-HajnalkaFulop RossWhitmore StephanieMcLennan  
title                                                               
zzDummy        1                  1            1                 1

In [49]:
prsn_no_jnls_df.sort_values(['position', 'name'])[['position', 'name']]

,position,name
11,Associate Investigator,AlejandroFontan Villacampa
4,Associate Investigator,EmmaCooper
9,Associate Investigator,FlaviaFerrari
27,Associate Investigator,FrancescoRicci
40,Associate Investigator,GavinWinter
7,Associate Investigator,GrantDuffy
14,Associate Investigator,JacintaHolloway-Brown
35,Associate Investigator,JohannaTurnbull
34,Associate Investigator,KateThompson
0,Associate Investigator,MehrdadAmirghasemi
